# Importações Necessárias

In [1]:
import numpy as np;
import pandas as pd;

# Parâmetros de Entrada

In [2]:
baseDados = "BPressureNishiBook.dat";
baseRegras = "BPressureNishiBook.txt";

# Pré-Processamento Base Transacional

In [167]:
# lendo arquivo
mt = pd.read_table(baseDados, delim_whitespace=True, dtype="str", header=None);
# pre-processando
mtBinaria = pd.get_dummies(mt);
dados = mtBinaria.astype('int').to_numpy();
print(dados);

[[1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0]
 [1 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 1]
 [0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1]
 [0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0]
 [0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 0]
 [0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0]
 [0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1]
 [1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1]
 [0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0]
 [1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1]
 [0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0]
 [0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 0 1 0]
 [0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0]
 [1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0]
 [0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 1 0]]


# Pré-processamento RAs

In [168]:
regras = pd.read_table(baseRegras, sep="#", names=("AR", "sup", "cnf"));
regras["sup"] = regras["sup"].str.replace("SUP:", "");
regras["cnf"] = regras["cnf"].str.replace("CONF:", "");

regras["itemset"] = regras["AR"].str.replace("==>", " ").str.split();

regras["antc"], regras["cons"] =  regras["AR"].str.split("==>").str;
regras["cons"] = regras["cons"].str.split();
regras["antc"] = regras["antc"].str.split();

print(regras);

                       AR  sup                   cnf                itemset  \
0                6 ==> 1    5     0.5555555555555556                 [6, 1]   
1                1 ==> 6    5                    1.0                 [1, 6]   
2                4 ==> 2    3                    1.0                 [4, 2]   
3                2 ==> 4    3                    0.5                 [2, 4]   
4                5 ==> 2    3                    1.0                 [5, 2]   
...                   ...  ...                   ...                    ...   
1369  15 ==> 3 6 9 12 16    1     0.3333333333333333  [15, 3, 6, 9, 12, 16]   
1370  12 ==> 3 6 9 15 16    1                    0.1  [12, 3, 6, 9, 15, 16]   
1371  9 ==> 3 6 12 15 16    1    0.16666666666666666  [9, 3, 6, 12, 15, 16]   
1372  6 ==> 3 9 12 15 16    1     0.1111111111111111  [6, 3, 9, 12, 15, 16]   
1373  3 ==> 6 9 12 15 16    1                   0.25  [3, 6, 9, 12, 15, 16]   

      antc                cons  
0      [6]        

# Suporte Absoluto

In [186]:
def abSuppRA(antc, consq):
    return calcSupp(list(antc) + list(consq));


def abSupp(itemset):
    base = np.copy(dados[:,itemset[0]-1]);

    for i in itemset:
        base *= dados[:,i-1];
   
    return np.sum(base, axis=0);
      
print("12 6 ==> 13 #SUPP: " + str(calcSuppRA([12, 6], [13]))) #5
print("12 17 14 ==> 2 #SUPP: " + str(calcSuppRA([12, 17, 14], [2]))) #2

12 6 ==> 13 #SUPP: 5
12 17 14 ==> 2 #SUPP: 2


# Suporte Relativo

In [180]:
def reSuppRA(antc, consq):
    return reSupp(list(antc) + list(consq));

def reSupp(itemset):
    return abSupp(itemset) / dados.shape[1];

# Confiança

In [185]:
def conf(antc,consq):
    return abSuppRA(antc,consq)/abSupp(antc);


print(conf([12, 17, 14], [2]));# 1.0
print(conf([6, 1, 18], [13])); # .666
print(conf([6, 13], [1, 18])); #0.2857142857142857

1.0
0.6666666666666666
0.2857142857142857


# Added Value

In [ ]:
def addedValue(antc, consq):
    return conf(antc, consq)-reSupp(consq);

# All-Confidence

In [ ]:
def allConf(itemset):
    maxSup = max([reSupp(i) for i in itemset])
    
    return reSupp(itemset)/maxSup